# Text als Daten

Natürlich sind Texte immer Daten, wenn sie die Grundlage wissenschaftlicher Analysen sind. Was ist hier also mit »Text als Daten« gemeint? Im Gegensatz zu vorherigen Einheiten, die sich an den sprachlichen Eingenschaften von Text orientieren, soll hier eine Betrachtungsweise von Text eingeführt werden, die stärker von der sprachlichen Gestalt abstrahiert. Aus Texten werden Tabellen und Zahlen und sie erhalten damit eine Form, die näher an den Datenstrukturen etwa der quantitativen Sozialforschung ist. Dies erlaubt dann Berechnungen auf den Daten, die neue Einblicke in die untersuchten Texte erlauben.

Als Ausgangspunkt soll wieder der Beispieltext aus der dritten Einheit dienen.

In [1]:
import codecs
from collections import Counter
from textblob_de import TextBlobDE as TextBlob

with codecs.open('../03_Sprache/Rede_Rundfunk.txt', 'r', 'utf-8') as infile:
    rede = infile.read()

blob = TextBlob(rede)

Zuvor hatten wir den Text in seinen sprachlichen Eigenschaften betrachtet: Bestehend aus Sätzen und Wörtern, die verschiedenen Arten wie Substantiven oder Verben zugeordnet werden können. Auf einer abstrakteren Ebene lässt sich ein Text aber auch einfach als Häufung von Worten betrachten. Ein Text lässt sich dann dadurch charakterisieren, welche Wörter in ihm wie häufig vorkommen. Dies wird als “bag of words”-Ansatz bezeichnet.

In [2]:
bag_of_words = Counter(blob.words)
bag_of_words.most_common(10)

[(u'der', 53),
 (u'und', 51),
 (u'die', 44),
 (u'auch', 39),
 (u'ist', 34),
 (u'in', 27),
 (u'wir', 24),
 (u'Sie', 24),
 (u'das', 22),
 (u'\u2013', 20)]

Will man nicht nur einen Text, sondern ein ganzes Corpus auf diese Art und Weise betrachten, bietet sich eine Tabellenform an: Die einzelnen Wörter sind dann die Variablen (Spalten), die Texte sind Fälle (Zeilen), und die Worthäufigkeiten sind Ausprägungen (Zellen).

Der Beispieltext ließe sich also auch entsprechend als Zeile in einer Tabelle darstellen:

In [3]:
import pandas as pd

words, counts = zip(*bag_of_words.most_common(20))

df = pd.DataFrame(columns=words)
df.loc['Rede Rundfunk'] = counts
df

,der,und,die,auch,ist,in,wir,Sie,das,–,für,sind,es,haben,den,zu,Rundfunk,des,wie,sich
Rede Rundfunk,53,51,44,39,34,27,24,24,22,20,19,18,18,17,17,16,16,15,15,15


Für eine solche Betrachtung von Textkorpora stellt das Python-Paket [gensim](http://radimrehurek.com/gensim/) ein Framework bereit. Ein `TextCorpus` ist in gensim eine Sequenz von Dokumenten, die jeweils aus Worthäufigkeiten bestehen. Gensim kann eine Reihe von in der Computerlinguistik üblichen Datenformaten einlesen und als `TextCorpus` bereitstellen. Wenn die Texte in einem anderen Format vorliegen, etwa als Ergebnis eines Web-Crawlings, kann relativ einfach ein eigener Import-Mechanismus definiert werden.

Unser Textcorpus liegt als CSV-Datei vor, bei der jede Zeile Metadaten zu einem Text und in der Spalte 'text' den eigentlichen Inhalt enthält. Um den Text aus einer CSV-Datei zu extrahieren, muss ein angepasster Corpus-Lese-Mechanismus gebaut werden. Dazu wird eine eigene Klasse definiert, die den Import übernimmt.

Auf der Python-Seite sind hier zwei Konzepte neu: Klassenvererbung und Generatoren. Ihre Details können in Python-Tutorials und der Python-Dokumentation nachgelesen werden, aber die für unsere Zwecke relevanten Informationen sollen hier noch einmal ins Gedächtnis gerufen werden.

## Vererbung

Die neu erstellte Klasse `CSVCorpus` erbt von der `TextCorpus`-Klasse aus gensim. Das bedeutet, dass unser Corpus wie jeder andere `TextCorpus` verwendet werden kann, also die gleichen Eigenschaften und Methoden besitzt. Nur die Teile, die in unserem Fall spezifisch sind, müssen überschrieben werden. Das ist in unserem Fall die Methode `get_texts()`. Sie definiert, wie die Texte eingelesen und verarbeitet werden.

Andere Methoden von `TextCorpus` können dabei verwendet werden, wie hier etwa `getstream()`. Diese Methode gibt die Datei zurück, in der das Corpus gespeichert ist.

## Generatoren

Generatoren wurden in der Einheit _Text als Netzwerk_ schon einmal angesprochen: Sie erlauben wie Listen eine Nutzung wie `for x in y`, aber nicht einen direkten Zugriff wie `y[i]`. Gensim nutzt diese Form, um nicht das gesamte Corpus auf einmal im Arbeitsspeicher halten zu müssen, was die Maximalgröße zu verarbeitender Corpora begrenzen würde. Statt dessen wird jeweils nur ein Dokument zur Zeit geladen und verarbeitet.

Einen Generator kann man selbst mit dem `yield`-Mechanismus erstellen: Damit werden nacheinander die entsprechenden Werte zurückgegeben. Gensim erwartet, dass jedes Dokument als Liste von Wörtern ausgegeben wird. Im Falle des Textcorpus wird also nacheinander jeder Text in der CSV-Datei über TextBlob in Wörter zerlegt und mit `yield` ausgegeben.

In [4]:
from gensim.corpora.textcorpus import TextCorpus

class CSVCorpus(TextCorpus):
    """Read corpus from a csv file."""

    def get_texts(self):
        with self.getstream() as csvfile:  # Öffnet die CSV-Datei
            table = pd.read_csv(csvfile, parse_dates=['date'], encoding='utf-8')  # Liest die CSV-Datei
            for text in table['text']:  # Verarbeite die einzelnen Texte aus der Spalte 'text'
                blob = TextBlob(text)
                yield blob.words

Um zu sehen, welche Form ein Dokument nun hat, kann man ein `CSVCorpus` mit der Datei `reden.csv` erstellen. Uns interessiert zunächst nur ein einzelnes Dokument. Da ein Zugriff in der Form `corpus[0]` bei einem Generator nicht möglich ist, kann man eine Schleife nutzen und nach dem ersten Durchlauf abbrechen. (Auch dieser Kniff wurde im Netzwerk-Notebook noch einmal detailliert erläutert.)

In [5]:
corpus = CSVCorpus('../Daten/reden.csv')
for doc in corpus:
    break
doc[0:20]

[(0, 1),
 (1, 1),
 (2, 1),
 (3, 1),
 (4, 1),
 (5, 1),
 (6, 1),
 (7, 1),
 (8, 1),
 (9, 1),
 (10, 1),
 (11, 2),
 (12, 1),
 (13, 1),
 (14, 1),
 (15, 1),
 (16, 1),
 (17, 1),
 (18, 4),
 (19, 1)]

Man sieht nun, dass das Dokument eine ähnliche Form hat wie die einfache `Counter`-Darstellung oben: Ein Dokument ist eine Liste aus Worthäufigkeiten. Es gibt dabei aber einen Unterschied: Statt der Worte selbst wird eine Zahl als Schlüssel für die Worte verwendet.

Man kann Corpora begrenzter Größe auch in der oben beschriebenen Tabellenform darstellen. Gensim stellt Funktionen bereit, die ein Corpus in eine Matrix umwandeln. Dabei unterscheidet man zwischen zwei Arten von Matrizen:

* Eine normale Matrix besteht aus Zeilen und Spalten. In der Regel, etwa in sozialwissenschaftlichen Datensätzen, hat jede Zelle einen Wert, es gibt nur vereinzelte fehlende Werte. Eine solche Matrix wird daher »dicht« (engl. »dense«) genannt.
* Bei einer Textmatrix gibt es so viele Spalten wie eindeutige Wörter im Corpus. Der Wortschatz eines einzelnen Textes ist aber in der Regel deutlich kleiner als der Wortschatz des Gesamtkorpus. Die meisten Zellen der Matrix bestehen daher aus Nullen. Eine solche Matrix nennt man »schwach besetzt« (engl. »sparse«). Einige Berechnungen sind für diese Art von Matrix optimiert.

Das Textcorpus kann also in eine *sparse matrix* konvertiert werden:

In [6]:
from gensim.matutils import corpus2csc

corpus_matrix_sparse = corpus2csc(corpus)
corpus_matrix_sparse.shape

(56570, 614)

Die Matrix hat 56570 Zeilen und 614 Spalten. Gensim hält sich hier also nicht an die Konvention, die einzelnen Wörter als Spalten aufzufassen, sondern stellt sie als Zeilen dar. Dies kann durch eine Transponierung leicht behoben werden:

In [7]:
corpus_matrix_sparse = corpus_matrix_sparse.transpose()
corpus_matrix_sparse.shape

(614, 56570)

Nun hat die Matrix für jedes Dokument eine Zeile. Der Gesamtwortschatz des Textes besteht folglich aus 56570 Wörtern.

Da ein `pandas.DataFrame` in IPython Notebooks hübscher ausgegeben wird als eine Matrix, kann man die Matrix entsprechend konvertieren. Da ein `DataFrame` aber nur aus einer dichten Matrix erzeugt werden kann, wird die Matrix als Zwischenschritt entsprechend umgewandelt.

In [8]:
corpus_matrix = corpus_matrix_sparse.todense()
df = pd.DataFrame(corpus_matrix[0:5,0:20])
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,1,1,1,1,1,1,1,1,1,1,1,2,1,1,1,1,1,1,4,1
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0
4,0,0,0,0,3,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0


Diese Darstellung ist natürlich nur begrenzt aussagekräftig, solange man nicht weiß, für welche Wörter die Spalten stehen. Gensim erzeugt für jedes Corpus ein Wörterbuch `corpus.dictionary`, das für jede Zahl das entsprechende Wort zurückgibt. Die numerischen Spaltenbezeichnungen lassen sich also in die jeweiligen Wörter übersetzen:

In [9]:
df.columns = [corpus.dictionary[i] for i in df.columns]
df

,100%igen,20,2006,750,Aber,Abgründe,Abgründen,Abgründigen,Affäre,Alice,Alltags,Als,Anekdoten,Anfang,Angesicht,Anmaßung,Ansprüche,Antlitz,Antworten,Anwandlungen
0,1,1,1,1,1,1,1,1,1,1,1,2,1,1,1,1,1,1,4,1
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0
4,0,0,0,0,3,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0


Die Metadaten der Texte, also etwa Titel oder Datum, sind im Corpus selbst nicht enthalten. Sie lassen sich aber aus der ursprünglichen CSV-Tabelle auslesen. Für eine aussagekräftigere Darstellung können also etwa die Dokumententitel als Zeilenbezeichnungen verwendet werden.

In [10]:
table = pd.read_csv('../Daten/reden.csv', parse_dates=['date'], encoding='utf-8')
titles = table['title']

df.index = [titles[i] for i in df.index]
df

,100%igen,20,2006,750,Aber,Abgründe,Abgründen,Abgründigen,Affäre,Alice,Alltags,Als,Anekdoten,Anfang,Angesicht,Anmaßung,Ansprüche,Antlitz,Antworten,Anwandlungen
Laudatio auf Rüdiger Safranski,1,1,1,1,1,1,1,1,1,1,1,2,1,1,1,1,1,1,4,1
Rede von Bundeskanzlerin Merkel anlässlich der Präsentation des deutsch-chinesischen Kooperationsprojekts zur Ladeinfrastruktur für Elektrofahrzeuge am 8. Juli 2014,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Rede von Bundeskanzlerin Merkel anlässlich des Besuchs der Tsinghua - Universität am 8. Juli 2014,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
"Kulturstaatsministerin Monika Grütters zur Eröffnung der Sonderausstellung\r\n""Flughafen Berlin-Tempelhof - Die amerikanische Geschichte""",0,1,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0
Rede von Bundeskanzlerin Merkel anlässlich der Taufe des Tiefseeforschungsschiffs „Sonne“ am 11. Juli 2014,0,0,0,0,3,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0


Diese Darstellung veranschaulicht das Prinzip, Texte als »bags of words« zu verstehen und Corpora als Matrizen. Die folgenden Analysemethoden basieren auf dieser Form von »Texten als Daten«.